In [1]:
from pymongo import MongoClient
import folium 
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import os

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db=client.proyecto2

In [3]:
datab = db.proyecto_coll2.find()
df = pd.DataFrame(datab)
df.shape # Comprobar filas y columnas del DF original

,_id,category_code,country_code,deadpooled_year,founded_year,lat,lng,name,number_of_employees,oficina_principal,total_money_raised
0,5d26fe94ded049a943505ad7,Empresa tecnologica,USA,None,2007,34.017606,-118.487267,Mahalo,40,"{'type': 'Point', 'coordinates': [-118.487267,...",$21M
1,5d26fe94ded049a943505ad8,Empresa tecnologica,USA,None,2006,34.090368,-118.393064,Geni,18,"{'type': 'Point', 'coordinates': [-118.393064,...",$16.5M
2,5d26fe94ded049a943505ad9,Empresa tecnologica,USA,None,2002,37.387845,-122.055197,Plaxo,50,"{'type': 'Point', 'coordinates': [-122.055197,...",$28.3M
3,5d26fe94ded049a943505ada,Empresa de videojuegos,USA,None,2006,37.788482,-122.409173,Kyte,40,"{'type': 'Point', 'coordinates': [-122.409173,...",$23.4M
4,5d26fe94ded049a943505adb,Empresa tecnologica,USA,None,2005,37.480999,-122.173887,Jingle Networks,35,"{'type': 'Point', 'coordinates': [-122.173887,...",$88.7M
5,5d26fe94ded049a943505adc,Empresa tecnologica,USA,None,2002,40.726040,-73.995722,Meetup,75,"{'type': 'Point', 'coordinates': [-73.995722, ...",$18.3M
6,5d26fe94ded049a943505add,Empresa tecnologica,USA,None,1984,37.408802,-121.953770,Cisco,63000,"{'type': 'Point', 'coordinates': [-121.95377, ...",$2.5M
7,5d26fe94ded049a943505ade,Empresa tecnologica,USA,None,1994,37.418531,-122.025485,Yahoo!,13600,"{'type': 'Point', 'coordinates': [-122.025485,...",$6.8M
8,5d26fe94ded049a943505adf,Empresa tecnologica,USA,None,1995,37.295005,-121.930035,eBay,15000,"{'type': 'Point', 'coordinates': [-121.930035,...",$6.7M
9,5d26fe94ded049a943505ae0,Empresa tecnologica,USA,None,2005,37.423390,-122.089951,Jajah,110,"{'type': 'Point', 'coordinates': [-122.0899512...",$33M


(848, 11)

In [4]:
db.proyecto_coll2.find()
# Empresas con una o más oficinas

In [5]:
# Se extrae una lista de geolocations
lista_find=[]
for i in range(len(df)):
    lista_find.append(df.oficina_principal[i])       

In [6]:
def findNear(lista_find, radio_max_meters):
# Funcion que busca las empresas cercanas en 'x' metros a partir de las coordenadas de una empresa
        geopoint=lista_find
        return list(db.proyecto_coll2.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    }))

radio_max_meters = 1000 

# Iteración para cada empresa con sus coordenadas
oficinas_cercanas = []
for i in range(len(df)):
    num_offices = findNear(lista_find[i], radio_max_meters)
    oficinas_cercanas.append(len(num_offices))

# Se agrega una nueva columna al DF que muestra el número de empresas en un radio de 1000m
df["Prox_ofis"] = oficinas_cercanas

In [7]:
def suma_empleados_cerca(df):
# Funcion para sumar la cantidad de empleados
    cnt=0
    for empresa in df:
        cnt+=empresa['number_of_employees']
    return cnt

In [8]:
col_sum_emp=[]
for i, e in df.iterrows():
# Se itera la funcion de sumar empleados y se aplica la GEOquery
    empresas_radio = findNear(lista_find[i], radio_max_meters)
    empl = suma_empleados_cerca(empresas_radio)
    col_sum_emp.append(empl)

In [9]:
# Se agrega una nueva columna
df["Suma_empl"] = col_sum_emp

In [10]:
# Se eliminan los registros de empresas duplicadas
df.drop_duplicates(subset='name', keep='first', inplace=True)

In [11]:
# Se filtra por un mínimo de 5 oficinas próximas
df = df[df.Prox_ofis > 4]

In [12]:
df["country_code"].value_counts()

USA    199
ISR      5
DNK      1
Name: country_code, dtype: int64

In [13]:
# Se convierte a listas las columnas de latitud y longitud para hacer queries a la API
lista_lats = df['lat'].tolist()
lista_longs = df['lng'].tolist()

In [14]:
# Token
load_dotenv()

google_k = os.environ["token_maps"]

In [15]:
# Se señalan los lugares que se van a buscar por la API
place_club="night_club"
place_bar="bar"
place_takeaway="meal_takeaway"

# Listas vacias para poder iterar en cada query y tener una lista sobre la que trabajar
lista_clubs=[]
lista_bar=[]
lista_takeaway=[]

# Queries a la API para encontrar night clubs, bares y takeaways en un radio de 1.000 metros
for i in range(len(lista_lats)):
    lista_clubs.append(requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type={}&key={}'.format(lista_lats[i], lista_longs[i], radio_max_meters, place_club, google_k)).json())
for i in range(len(lista_lats)):
    lista_bar.append(requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type={}&key={}'.format(lista_lats[i], lista_longs[i], radio_max_meters, place_bar, google_k)).json())
for i in range(len(lista_lats)):
    lista_takeaway.append(requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type={}&key={}'.format(lista_lats[i], lista_longs[i], radio_max_meters, place_takeaway, google_k)).json())

In [16]:
# Se crea una lista con la suma de clubs por cada empresa 
suma_clubs=[]
for i in range(len(lista_clubs)): 
    suma_clubs.append(len(lista_clubs[i]['results']))

In [17]:
# Se añade una columna al DF
df['Total_clubs']=suma_clubs

In [18]:
# Se crea una lista con la suma de bares por cada empresa 
suma_bar=[]
for i in range(len(lista_bar)):
    suma_bar.append(len(lista_bar[i]['results']))

In [19]:
# Se añade una columna al DF
df['Total_bar']=suma_bar

In [20]:
# Se crea una lista con la suma de takeaways por cada empresa
suma_takeaway=[]
for i in range(len(lista_takeaway)):
    suma_takeaway.append(len(lista_takeaway[i]['results']))

In [21]:
# Se añade una columna al DF
df['Total_takeaway']=suma_takeaway

In [22]:
df.head()

,_id,category_code,country_code,deadpooled_year,founded_year,lat,lng,name,number_of_employees,oficina_principal,total_money_raised,Prox_ofis,Suma_empl,Total_clubs,Total_bar,Total_takeaway
3,5d26fe94ded049a943505ada,Empresa de videojuegos,USA,None,2006,37.788482,-122.409173,Kyte,40,"{'type': 'Point', 'coordinates': [-122.409173,...",$23.4M,30,2402,20,20,20
5,5d26fe94ded049a943505adc,Empresa tecnologica,USA,None,2002,40.726040,-73.995722,Meetup,75,"{'type': 'Point', 'coordinates': [-73.995722, ...",$18.3M,10,8617,20,20,20
9,5d26fe94ded049a943505ae0,Empresa tecnologica,USA,None,2005,37.423390,-122.089951,Jajah,110,"{'type': 'Point', 'coordinates': [-122.0899512...",$33M,8,9081,0,1,2
10,5d26fe94ded049a943505ae1,Empresa de videojuegos,USA,None,2007,37.392936,-122.079480,Ustream,250,"{'type': 'Point', 'coordinates': [-122.07948, ...",$60.1M,13,1960,6,20,4
11,5d26fe94ded049a943505ae2,Empresa de videojuegos,USA,None,2007,40.726155,-73.995625,Livestream,120,"{'type': 'Point', 'coordinates': [-73.995625, ...",$14.7M,10,8617,20,20,20


In [23]:
# Se crea un ranking en funcion de la suma de 
# empleados totales que se encuentran en un radio de 1.000 metros de cada empresa
df['ranked_networking']=df['Suma_empl'].rank(ascending=True)

In [24]:
# En base a este primer ranking se agregan valoraciones segun el total de clubs, bares y sitios de takeaway.
# Además de sumar los locales cercanos, se suma la categoría, dandole mas valor a las empresas de videojuegos
df["ranking_final"] = df["ranked_networking"] + df["Total_clubs"]*0.1 + df['Total_bar']*0.1 + df['Total_takeaway']*0.1 + np.where(df['category_code'] == 'Empresa de videojuegos', 30, 0)

In [25]:
# Se ordena el DF por el ranking y se seleccionan las 10 primeras empresas
df.sort_values(['ranking_final'], inplace = True, ascending=False)
df.reset_index()
df_top10=df.iloc[0:10, 1:]

In [26]:
df_top10

,category_code,country_code,deadpooled_year,founded_year,lat,lng,name,number_of_employees,oficina_principal,total_money_raised,Prox_ofis,Suma_empl,Total_clubs,Total_bar,Total_takeaway,ranked_networking,ranking_final
11,Empresa de videojuegos,USA,None,2007,40.726155,-73.995625,Livestream,120,"{'type': 'Point', 'coordinates': [-73.995625, ...",$14.7M,10,8617,20,20,20,203.5,239.5
251,Empresa de videojuegos,USA,None,2005,40.739930,-73.993049,Cellufun,30,"{'type': 'Point', 'coordinates': [-73.9930489,...",$5.23M,13,8490,20,20,20,201.0,237.0
399,Empresa de videojuegos,USA,None,2000,40.738476,-73.998726,VISUALPLANT,6,"{'type': 'Point', 'coordinates': [-73.998726, ...",$500k,9,8212,20,20,20,198.0,234.0
40,Empresa de videojuegos,USA,None,2006,37.787092,-122.399972,Curse,58,"{'type': 'Point', 'coordinates': [-122.399972,...",$12M,50,3708,20,20,20,196.5,232.5
112,Empresa de videojuegos,USA,None,0,37.783898,-122.395234,Rupture,25,"{'type': 'Point', 'coordinates': [-122.395234,...",$3M,49,3624,20,20,20,194.0,230.0
61,Empresa de videojuegos,USA,None,2006,37.786942,-122.401245,Kongregate,20,"{'type': 'Point', 'coordinates': [-122.401245,...",$19M,48,3108,20,20,20,185.0,221.0
35,Empresa de videojuegos,USA,None,2003,37.788668,-122.400558,hi5,100,"{'type': 'Point', 'coordinates': [-122.400558,...",$52M,46,3005,20,20,20,178.0,214.0
291,Empresa de videojuegos,USA,None,2008,37.789321,-122.401362,Serious Business,22,"{'type': 'Point', 'coordinates': [-122.4013624...",$4M,44,2986,20,20,20,176.0,212.0
5,Empresa tecnologica,USA,None,2002,40.726040,-73.995722,Meetup,75,"{'type': 'Point', 'coordinates': [-73.995722, ...",$18.3M,10,8617,20,20,20,203.5,209.5
693,Empresa tecnologica,USA,None,2007,40.738567,-73.987199,SpaBooker,42,"{'type': 'Point', 'coordinates': [-73.987199, ...",$14.7M,14,8499,20,20,20,202.0,208.0


In [27]:
# Tras comprobar que todo esta en orden se selecciona la mejor puntuada y se marca en el mapa
df_final=df.iloc[0:1, :]

In [28]:
# Con Follium se marca la empresa
mapa_1 = folium.Map(location=[40.726155, -73.995625], width=900, height=500, zoom_start=2,)
for index, row in df_final.iterrows():
    folium.CircleMarker([row['lat'], row['lng']],
                        radius=5,
                        popup="Name: {} Categoría: {}".format(row['name'],row["category_code"]),
                        fill_color="#3db7e4", # divvy color
                       ).add_to(mapa_1)


In [29]:
mapa_1

In [30]:
# La coordenada final es: [40.726155, -73.995625]